In [9]:
import os
import json
import requests
from openai import OpenAI
import time
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv())

FINANCIAL_URL = os.environ.get("FINANCIAL_URL")
FMP_API_KEY = os.environ.get("FMP_API_KEY")
MODEL = "gpt-3.5-turbo-1106"

client = OpenAI()

In [14]:
# fucntions for fetching the financial data
# Define financial statement functions

def get_income_statement(ticker:str, period:str, limit:int) -> json:
    url = f"{FINANCIAL_URL}/income-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    print('response')
    print(response.json())
    
    return json.dumps(response.json())

def get_balance_sheet(ticker, period, limit):
    # Code to fetch and return cash flow statement
    pass

def get_cash_flow_statement(ticker, period, limit):
    # Code to fetch and return cash flow statement
    pass

def get_key_metrics(ticker, period, limit):
    # Code to fetch and return cash flow statement
    pass

def get_financial_ratios(ticker, period, limit):
    # Code to fetch and return cash flow statement
    pass

def get_financial_growth(ticker, period, limit):
    # Code to fetch and return cash flow statement
    pass


availableFunctions:dict = {
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_cash_flow_statement": get_cash_flow_statement,
    "get_key_metrics": get_key_metrics,
    "get_financial_ratios": get_cash_flow_statement,
    "get_financial_growth": get_financial_ratios
}





In [29]:
# run function

def run_assistant(user_message):
    assistant = client.beta.assistants.create(
        instructions="Act as a financial analyst by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends. ",
        model=MODEL,
        tools=[
            {
                "type": "function", 
                "function": {
                    "name": "get_income_statement", 
                    "parameters": {
                        "type": "object", 
                        "properties": {
                            "ticker": {"type": "string"}, 
                            "period": {"type": "string"}, 
                            "limit": {"type": "integer"}
                        }
                    }
                }
            },
            # same for the rest of the financial functions
          ]     
        )
    
    # creating thread
    thread = client.beta.threads.create()
    
    # creating messages
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_message
    )

    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)
    

    # Loop until the run completes or requires action
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

        # Add run steps retrieval here
        run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
        print("Run Steps:", run_steps)

        if run.status == "requires_action":
            
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                if function_name in availableFunctions:
                    function_to_call = availableFunctions[function_name]
                    output = function_to_call(**function_args)
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": output,
                    })

            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )

        elif run.status == "completed":
            # List the messages to get the response
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            for message in messages.data:
                role_label = "User" if message.role == "user" else "Assistant"
                message_content = message.content[0].text.value
                print(f"{role_label}: {message_content}\n")
            break  # Exit the loop after processing the completed run

        elif run.status == "failed":
            print("Run failed.")
            break

        elif run.status in ["in_progress", "queued"]:
            print(f"Run is {run.status}. Waiting...")
            time.sleep(1)  # Wait for 5 seconds before checking again

        else:
            print(f"Unexpected status: {run.status}")
            break


In [30]:
run_assistant("Can you compare the financial health of Microsoft and Apple over the last 1 years, focusing on their balance sheets and key financial ratios?")

Run Steps: SyncCursorPage[RunStep](data=[], object='list', first_id=None, last_id=None, has_more=False)
Run is in_progress. Waiting...
Run Steps: SyncCursorPage[RunStep](data=[RunStep(id='step_ILsMCwTTaW0SwFWegJj8119W', assistant_id='asst_q64XdTr2Fl5TpbUSBUlCcbaA', cancelled_at=None, completed_at=None, created_at=1706720995, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_kxIZONhSEZBeaQU7ZXSYt7wH', status='in_progress', step_details=ToolCallsStepDetails(tool_calls=[], type='tool_calls'), thread_id='thread_2c13t1fDWReNCXkGuNKWu7gI', type='tool_calls', usage=None, expires_at=1706721593)], object='list', first_id='step_ILsMCwTTaW0SwFWegJj8119W', last_id='step_ILsMCwTTaW0SwFWegJj8119W', has_more=False)
Run is in_progress. Waiting...
Run Steps: SyncCursorPage[RunStep](data=[RunStep(id='step_ILsMCwTTaW0SwFWegJj8119W', assistant_id='asst_q64XdTr2Fl5TpbUSBUlCcbaA', cancelled_at=None, completed_at=None, created_at=1706720995, expired_at=Non

In [22]:
print(messages)

NameError: name 'messages' is not defined